In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
os.environ['TOKEN'] = "AAAAAAAAAAAAAAAAAAAAAAs0ggEAAAAAZa1IS2xF3DqB4kIElAFDyc6oyZM%3DKaJowQh3hk4sqf8b8Lb3DrlffIBE7wDWoTrbTQNrGI4tbaqXRz"

In [3]:
def auth():
    return os.getenv('TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, max_results = 50):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [13]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "Elon Musk lang:en"
start_time = "2022-01-01T00:00:00.000Z"
end_time = "2021-08-29T00:00:00.000Z"
max_results = 50

In [14]:
url = create_url(keyword)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [15]:
tweets = []
for i in range((50)):
    try:
        tweets.append(json_response["data"][i]["text"])
    except IndexError:
        break



In [16]:
tweets

['RT @mazemoore: “Twitter is so hateful ever since Elon Musk took over.” 🤪',
 'RT @w_terrence: Whoopi Goldberg announced she has left Twitter because of Elon Musk. I just want to thank Elon for doing such a good job at…',
 'RT @donmoyn: Turns out that the Content Moderation Council was just Andy Ngo telling Elj Musk which left wing accounts to ban https://t.co/…',
 'RT @RT_com: Elon Musk has said he anticipates a wireless brain chip developed by his company Neuralink to start human clinical testing in s…',
 'RT @CL4WS_OUT: “Elon Musk is literally sitting on a dossier.” ~ Dave Rubin @RubinReport 🔥🔥🔥',
 "RT @GeneRSchmidt: @thetoyman1 Tim Cook is a leftist/globalist snake. The only reason he will not ban Twitter from App Store is he's worried…",
 'RT @itsJeffTiedrich: your reminder that Elon Musk is not an engineer, Donald Trump is not a successful businessman and Ron DeSantis is not…',
 "They Call It 'The Big Bang' – Elon Musk's Twitter Team Begins Process Of Reinstating 62,000 Suspended

In [17]:
pd.DataFrame(tweets)

,0
0,RT @mazemoore: “Twitter is so hateful ever sin...
1,RT @w_terrence: Whoopi Goldberg announced she ...
2,RT @donmoyn: Turns out that the Content Modera...
3,RT @RT_com: Elon Musk has said he anticipates ...
4,RT @CL4WS_OUT: “Elon Musk is literally sitting...
5,RT @GeneRSchmidt: @thetoyman1 Tim Cook is a le...
6,RT @itsJeffTiedrich: your reminder that Elon M...
7,They Call It 'The Big Bang' – Elon Musk's Twit...
8,RT @bennyjohnson: Jim Jordan takes flamethrowe...
9,RT @BSDetector4u: Elon Musk has triggered prac...


In [18]:
import nltk

#nltk.download("vader_lexicon")

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [19]:
vds = SentimentIntensityAnalyzer()
text = tweets[0]

print(text)

print("Sentiment Analysis of text")
print(vds.polarity_scores(text))

RT @mazemoore: “Twitter is so hateful ever since Elon Musk took over.” 🤪
Sentiment Analysis of text
{'neg': 0.272, 'neu': 0.728, 'pos': 0.0, 'compound': -0.6269}


In [20]:
for i in tweets:
    print("Tweet: ", i)
    print("Score: ", vds.polarity_scores(i))
    print("********")

Tweet:  RT @mazemoore: “Twitter is so hateful ever since Elon Musk took over.” 🤪
Score:  {'neg': 0.272, 'neu': 0.728, 'pos': 0.0, 'compound': -0.6269}
********
Tweet:  RT @w_terrence: Whoopi Goldberg announced she has left Twitter because of Elon Musk. I just want to thank Elon for doing such a good job at…
Score:  {'neg': 0.0, 'neu': 0.758, 'pos': 0.242, 'compound': 0.6908}
********
Tweet:  RT @donmoyn: Turns out that the Content Moderation Council was just Andy Ngo telling Elj Musk which left wing accounts to ban https://t.co/…
Score:  {'neg': 0.141, 'neu': 0.859, 'pos': 0.0, 'compound': -0.5574}
********
Tweet:  RT @RT_com: Elon Musk has said he anticipates a wireless brain chip developed by his company Neuralink to start human clinical testing in s…
Score:  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
********
Tweet:  RT @CL4WS_OUT: “Elon Musk is literally sitting on a dossier.” ~ Dave Rubin @RubinReport 🔥🔥🔥
Score:  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
***